In [1]:
### 중요: 이 코드는 GPT 모델 학습을 위한 데이터셋 클래스를 정의합니다

import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        """
        Args:
            txt (str): 학습할 전체 텍스트 데이터
            tokenizer: 텍스트를 토큰 ID로 변환해주는 토크나이저 (예: tiktoken)
            max_length (int): 모델이 한 번에 볼 수 있는 윈도우 크기 (입력 시퀀스 길이)
            stride (int): 윈도우를 이동시킬 간격 (데이터 중복 정도를 결정)
        """
        self.input_ids = []
        self.target_ids = []

        # 1. 전체 텍스트 토큰화
        # 텍스트를 정수 리스트(token_ids)로 변환합니다.
        # <|endoftext|> 같은 특수 토큰도 허용하여 인코딩합니다.
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # 데이터가 너무 짧으면 학습할 수 없으므로 최소 길이를 확인합니다.
        assert len(token_ids) > max_length, "토큰화된 입력의 개수는 적어도 max_length+1과 같아야 합니다."

        # 2. 슬라이딩 윈도우(Sliding Window)로 데이터 생성
        # 전체 토큰 리스트를 훑으며 max_length 길이만큼 잘라냅니다.
        # stride만큼 건너뛰며 반복합니다.
        for i in range(0, len(token_ids) - max_length, stride):
            
            # 입력 청크: 현재 위치(i)부터 max_length만큼 가져옵니다.
            input_chunk = token_ids[i : i + max_length]
            
            # 타겟 청크: 입력보다 1칸 뒤의 위치(i+1)부터 가져옵니다.
            # GPT는 '다음 단어'를 맞추는 모델이므로, 정답은 입력보다 한 칸씩 뒤로 밀려있어야 합니다.
            target_chunk = token_ids[i + 1 : i + max_length + 1]
            
            # 추출한 데이터를 텐서(Tensor)로 변환하여 리스트에 저장합니다.
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        # 데이터셋의 총 샘플(청크) 개수를 반환합니다.
        return len(self.input_ids)

    def __getitem__(self, idx):
        # DataLoader가 데이터를 요청할 때 호출됩니다.
        # 해당 인덱스(idx)의 입력과 정답 쌍을 반환합니다.
        return self.input_ids[idx], self.target_ids[idx]

In [2]:
##중요 : 아래 함수는 DataLoader를 생성하는 헬퍼 함수입니다.
import tiktoken
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # 토크나이저를 초기화합니다.
    tokenizer = tiktoken.get_encoding("gpt2")

    # 데이터셋을 만듭니다.
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # 데이터 로더를 만듭니다.
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [3]:
with open("datas/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)

first_batch = next(data_iter)
print(first_batch)

second_batch = next(data_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


## Tokenizer 비교

- 아래 결과를 보면 한국어는 Exaone, 중국어는 Qwen의 Tokenizer 성능이 좋습니다

In [ ]:
from transformers import AutoTokenizer
# Tokenizer 로드 (로컬 또는 온라인에서)
print("Tokenizer 로드 중...")
qwen = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")
exaone = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-4.0-1.2B")

# 테스트할 텍스트
texts = [
    "안녕하세요, 반갑습니다!",
    "Hello, nice to meet you!",
    "인공지능 기술이 발전하고 있습니다.",
    "Hello! This is an English tokenizer test.",
    "こんにちは！日本語のテストです。",
    "人工智能技术正在快速发展。",
    "The quick brown fox jumps over the lazy dog. 빠른 갈색 여우가 게으른 개를 뛰어넘습니다.",
    "파이썬 프로그래밍은 매우 유용합니다. Python programming is very useful.",
    "123456789 !@#$%^&*() 특수문자",
]

print("\n" + "="*60)
for text in texts:
    print(f"\n텍스트: {text}")
    print("-"*60)
    
    # Qwen3
    qwen_tokens = qwen.encode(text, add_special_tokens=True)
    qwen_token_strs = [qwen.decode([tok]) for tok in qwen_tokens[:10]]
    print(f"Qwen3:  {len(qwen_tokens)}개 토큰")
    print(f"        {qwen_token_strs}")
    
    # EXAONE
    exaone_tokens = exaone.encode(text, add_special_tokens=True)
    exaone_token_strs = [exaone.decode([tok]) for tok in exaone_tokens[:10]]
    print(f"EXAONE: {len(exaone_tokens)}개 토큰")
    print(f"        {exaone_token_strs}")
    
    # 비교
    diff = len(qwen_tokens) - len(exaone_tokens)
    if diff > 0:
        print(f"→ EXAONE이 {diff}개 더 효율적")
    elif diff < 0:
        print(f"→ Qwen3가 {abs(diff)}개 더 효율적")
    else:
        print(f"→ 동일")

    # Special tokens 비교
print("\n" + "=" * 80)
print("4. Special Tokens 비교")
print("=" * 80)

print(f"\n[Qwen3 Special Tokens]")
for token_name in ['bos_token', 'eos_token', 'unk_token', 'pad_token']:
    token = getattr(qwen, token_name, None)
    if token:
        token_id = getattr(qwen, f"{token_name}_id", None)
        print(f"  {token_name}: '{token}' (ID: {token_id})")

print(f"\n[EXAONE Special Tokens]")
for token_name in ['bos_token', 'eos_token', 'unk_token', 'pad_token']:
    token = getattr(exaone, token_name, None)
    if token:
        token_id = getattr(exaone, f"{token_name}_id", None)
        print(f"  {token_name}: '{token}' (ID: {token_id})")

## Hugging Face 모델 파일 구조 설명

Hugging Face Hub에 업로드된 파일들은 **Transformer 기반 언어 모델(LLM)**을 구성하는 핵심 요소들입니다. 크게 **설계도(Config)**, **가중치(Weights)**, **토크나이저(Tokenizer)** 세 그룹으로 나뉩니다.

---

### 1. 모델 아키텍처 및 설정 (설계도)

* **`config.json`**
    * **의미:** 모델의 **구조와 설계도**가 담긴 가장 중요한 설정 파일입니다.
    * **내용:** 모델의 종류(Llama, BERT 등), 레이어(Layer) 수, 히든 사이즈, 어텐션 헤드 수 등을 정의하여 빈 모델(Skeleton)을 만듭니다.

* **`generation_config.json`**
    * **의미:** 텍스트 **생성(Generation) 시 사용할 기본 옵션** 파일입니다.
    * **내용:** `max_length`(최대 길이), `temperature`(창의성), `top_p`, `eos_token_id`(종료 토큰) 등의 기본 설정값이 들어있습니다.

### 2. 모델 가중치 (학습된 뇌)

* **`model.safetensors`**
    * **의미:** 실제 **학습된 파라미터(가중치)**가 저장된 파일입니다. (가장 용량이 큼)
    * **특징:**
        * 기존 PyTorch의 `.bin` 파일보다 로딩 속도가 빠릅니다.
        * 보안 취약점(악성 코드 실행 등)을 해결한 안전한 형식입니다.

### 3. 토크나이저 (언어 처리기)
사람의 언어(Text)를 모델이 이해하는 숫자(Token ID)로 변환하는 규칙들입니다.

* **`vocab.json`**
    * **의미:** **단어장(Vocabulary)** 파일입니다. 단어와 그에 대응하는 숫자 ID가 매핑되어 있습니다.
* **`merges.txt`**
    * **의미:** **BPE 병합 규칙** 파일입니다. 알파벳 조각들이 어떻게 합쳐져서 단어가 되는지 정의합니다.
* **`tokenizer.json`**
    * **의미:** `vocab.json`과 `merges.txt` 등을 하나로 통합하여 최적화한 파일로, 로딩 속도가 매우 빠릅니다.
* **`tokenizer_config.json`**
    * **의미:** 토크나이저 설정 파일입니다. 어떤 클래스를 쓸지, 특수 토큰(`<s>`, `<pad>` 등)은 무엇인지 정의합니다.

---

### 요약

| 파일명 | 비유 | 역할 |
| :--- | :--- | :--- |
| `config.json` | **설계도** | 모델 구조 정의 |
| `model.safetensors` | **뇌** | 학습된 지식(가중치) 저장 |
| `tokenizer.json` | **번역기** | 텍스트 ↔ 숫자 변환 |
| `generation_config.json` | **말하기 습관** | 생성 옵션 설정 |